# Storing and using Secrets for your labs and demos

You can use the Microsoft.PowerShell.SecretManagement module introduced here to store and retrieve secrets with PowerShell.

Installation
Follow this notebook [Secrets Management with PowerShell](Secrets_Management_with_PowerShell.ipynb) for installation instructions

## Store your secrets

We can use the built in local vault to store secrets for labs and demos. You can add your secrets using a couple of methods described in the first notebook

In [7]:
Set-Secret -Name BeardLabSaAccount -Secret 'BeardsAreAwesome!'

Look at [Secrets Management with PowerShell](Secrets_Management_with_PowerShell.ipynb) notebook for more options

## Using secrets in your code

Now you can use the secrets in your code. Lets create a docker container and use the SA password from the vault

In [13]:
$DockerSA = Get-Secret -Name BeardLabSaAccount -AsPlainText
docker run -e "ACCEPT_EULA=Y" -e "SA_PASSWORD=$DockerSA" -p 15999:1433 --name beardcontainer -d mcr.microsoft.com/mssql/server:2019-latest

1903468df1adcd740b405d97e23c26678219a49f0302fcf10341bd2e4cbf02bf


Now we can query the container with dbatools by retrieving the password from the vault and creating a credential object 

In [19]:
$DockerSaCred = New-Object System.Management.Automation.PSCredential ('sa',(Get-Secret -Name BeardLabSaAccount))
$ContainerSQL = Connect-DbaInstance -SqlInstance 'localhost,15999' -SqlCredential $DockerSaCred

Get-DbaDatabase -SqlInstance $containersql |Format-Table
Get-DbaLogin -SqlInstance $containersql | Format-Table

ComputerName InstanceName SqlInstance  Name   Status IsAccessible RecoveryModel LogReuseWaitStatus


------------ ------------ -----------  ----   ------ ------------ ------------- ------------------


localhost    MSSQLSERVER  135b32b4a432 master Normal         True        Simple        Transaction


localhost    MSSQLSERVER  135b32b4a432 tempdb Normal         True        Simple            Nothing


localhost    MSSQLSERVER  135b32b4a432 model  Normal         True          Full            Nothing


...

localhost    MSSQLSERVER  135b32b4a432 ##MS_PolicyTsqlExecutionLogin##       SqlLogin 10/06/2020 1…


localhost    MSSQLSERVER  135b32b4a432 BUILTIN\Administrators            WindowsGroup 10/06/2020 1…


localhost    MSSQLSERVER  135b32b4a432 NT AUTHORITY\NETWORK SERVICE       WindowsUser 18/07/2020 0…


localhost    MSSQLSERVER  135b32b4a432 NT AUTHORITY\SYSTEM                WindowsUser 18/07/2020 0…


localhost    MSSQLSERVER  135b32b4a432 sa                                    SqlLogin 08/04/2003 0…


## Remove the container

In [12]:
docker rm beardcontainer --force
docker ps -a

beardcontainer


CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES
